In [ ]:
# Mount your google drive to import dataset 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# View total of dataset, customize the path according to your path
import os

jalan_dir = os.path.join('/content/drive/MyDrive/Image Detection/data/train/jalan')
kebakaran_dir = os.path.join('/content/drive/MyDrive/Image Detection/data/train/kebakaran')
pohon_dir = os.path.join('/content/drive/MyDrive/Image Detection/data/train/pohon')

print('total training jalan images:', len(os.listdir(jalan_dir)))
print('total training kebakaran images:', len(os.listdir(kebakaran_dir)))
print('total training pohon images:', len(os.listdir(pohon_dir)))


total training jalan images: 500
total training kebakaran images: 500
total training pohon images: 500


In [ ]:
# Import dataset
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/content/drive/MyDrive/Image Detection/data/train"
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

VALIDATION_DIR = "/content/drive/MyDrive/Image Detection/data/test"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=10
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=10
)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}):
        if(logs.get('loss')<0.001):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True
            
callbacks = myCallback()

Found 1499 images belonging to 3 classes.
Found 449 images belonging to 3 classes.


In [ ]:
# Model
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=50, steps_per_epoch=30, validation_data = validation_generator, verbose = 1, validation_steps=3, callbacks=[callbacks])

model.save("model.h5")

Epoch 1/50
30/30 [==============================] - 187s 6s/step - loss: 1.0366 - accuracy: 0.4767 - val_loss: 1.4556 - val_accuracy: 0.5333
Epoch 2/50
30/30 [==============================] - 148s 5s/step - loss: 1.0737 - accuracy: 0.5933 - val_loss: 0.9980 - val_accuracy: 0.5333
Epoch 3/50
30/30 [==============================] - 118s 4s/step - loss: 0.7160 - accuracy: 0.6167 - val_loss: 0.8842 - val_accuracy: 0.6000
Epoch 4/50
30/30 [==============================] - 101s 3s/step - loss: 0.7070 - accuracy: 0.7133 - val_loss: 1.2149 - val_accuracy: 0.5000
Epoch 5/50
30/30 [==============================] - 82s 3s/step - loss: 0.7874 - accuracy: 0.6722 - val_loss: 0.6153 - val_accuracy: 0.6000
Epoch 6/50
30/30 [==============================] - 74s 2s/step - loss: 0.6881 - accuracy: 0.7567 - val_loss: 0.4820 - val_accuracy: 0.9000
Epoch 7/50
30/30 [==============================] - 62s 2s/step - loss: 0.6221 - accuracy: 0.7759 - val_loss: 0.6998 - val_accuracy: 0.8000
Epoch 8/50
30/30

In [ ]:
# Load saved model

model = tf.keras.models.load_model('/content/drive/MyDrive/Image Detection/data2/model/model.h5')

# Check its architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [ ]:
# Do inference from URL

import urllib.request
from PIL import Image

urllib.request.urlretrieve(
  "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTzfNVD-N3441Afa4abUekCoj5Zl9uJjCRoHeKP0elAQY32yqI25me3OXQ&s=10",
   "gfg.png")
  
img=image.load_img("gfg.png", target_size=(150, 150))

x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images, batch_size=10)
  
print(classes[0])

[0. 0. 1.]


In [ ]:
# Do inference from local file

import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
   path='/content/' + fn
   img=image.load_img(path, target_size=(150, 150))
  
   x=image.img_to_array(img)
   x=np.expand_dims(x, axis=0)
   images = np.vstack([x])
  
   classes = model.predict(images, batch_size=10)
  
print(classes[0])

Saving kebakaran.jpeg to kebakaran.jpeg
[0. 1. 0.]


In [ ]:
# See class list

print(train_generator.class_indices)
print(validation_generator.class_indices)

{'jalan': 0, 'kebakaran': 1, 'pohon': 2}
{'jalan': 0, 'kebakaran': 1, 'pohon': 2}
